In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from backend.football_backend_client import fetch_upcoming_fixtures

In [6]:
upcoming_fixtures = fetch_upcoming_fixtures()
print(upcoming_fixtures)

      Home_Team_Name     Away_Team_Name
0          Brentford            Arsenal
1            Burnley           West Ham
2            Everton  Manchester United
3             Fulham             Wolves
4              Luton     Crystal Palace
..               ...                ...
255   Crystal Palace        Aston Villa
256        Liverpool             Wolves
257            Luton             Fulham
258  Manchester City           West Ham
259    Sheffield Utd          Tottenham

[260 rows x 2 columns]


In [7]:
df = pd.read_csv('../data/Premier_League_all_match_stats_2015-2023.csv')

In [8]:
label_encoder = LabelEncoder()
df['Home_Team_Name'] = label_encoder.fit_transform(df['Home_Team_Name'])
df['Away_Team_Name'] = label_encoder.transform(df['Away_Team_Name'])

upcoming_fixtures['Home_Team_Name'] = label_encoder.fit_transform(upcoming_fixtures['Home_Team_Name'])
upcoming_fixtures['Away_Team_Name'] = label_encoder.transform(upcoming_fixtures['Away_Team_Name'])

df['Home_passes_%'] = df['Home_passes_%'].str.rstrip('%').astype('float') / 100.0
df['Away_passes_%'] = df['Away_passes_%'].str.rstrip('%').astype('float') / 100.0
df['Home_ball_possession'] = df['Home_ball_possession'].str.rstrip('%').astype('float') / 100.0
df['Away_ball_possession'] = df['Away_ball_possession'].str.rstrip('%').astype('float') / 100.0

df = df.fillna(0)

df = df.drop(['Home_expected_goals', 'Away_expected_goals', 'Away_blocked_shots', 'Away_goalkeeper_saves', 'Away_offsides', 'Away_passes_%', 'Season', 'Match_Date', 'Away_passes_accurate', 'Away_shots_insidebox', 'Away_shots_off_goal', 'Away_shots_on_goal', 'Away_shots_outsidebox', 'Home_blocked_shots', 'Home_goalkeeper_saves', 'Home_offsides', 'Home_passes_%', 'Home_passes_accurate', 'Away_total_passes', 'Home_shots_insidebox', 'Home_shots_off_goal', 'Home_shots_on_goal', 'Home_shots_outsidebox', 'Home_total_passes'], axis=1)

In [9]:
target_columns = ['Home_Goals', 'Away_Goals',
       'Home_total_shots', 'Home_corner_kicks', 'Home_ball_possession',
       'Home_yellow_cards', 'Away_total_shots', 'Away_corner_kicks',
       'Away_ball_possession', 'Away_yellow_cards', 'Away_red_cards',
       'Home_red_cards', 'Home_fouls', 'Away_fouls']

In [10]:
models = {}

for target in target_columns:
    features = df.loc[:, df.columns != target]
    X = features
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    models[target] = model

    mse = mean_squared_error(y_test, model.predict(X_test))
    print(f'Mean Squared Error for {target}: {mse}')

Mean Squared Error for Home_Goals: 1.3989319623183698
Mean Squared Error for Away_Goals: 1.0523199216426324
Mean Squared Error for Home_total_shots: 15.574763984842757
Mean Squared Error for Home_corner_kicks: 5.9364432897932735
Mean Squared Error for Home_ball_possession: 1.051644383162341e-30
Mean Squared Error for Home_yellow_cards: 1.2237692016958917
Mean Squared Error for Away_total_shots: 13.524598451265906
Mean Squared Error for Away_corner_kicks: 5.112634381743198
Mean Squared Error for Away_ball_possession: 4.660603854833109e-31
Mean Squared Error for Away_yellow_cards: 1.4758893885114426
Mean Squared Error for Away_red_cards: 0.04619326850329754
Mean Squared Error for Home_red_cards: 0.06529431983490788
Mean Squared Error for Home_fouls: 13.047995892418303
Mean Squared Error for Away_fouls: 13.596075513081864


In [11]:
for index, row in upcoming_fixtures.iterrows():
    home_team = label_encoder.inverse_transform([row['Home_Team_Name']])[0]
    away_team = label_encoder.inverse_transform([row['Away_Team_Name']])[0]
    teams_involved = f"{home_team} vs {away_team}"
    print(f"\nTeams: {teams_involved}")

    for target in target_columns:
        row_copy = row.copy()
        
        for col, val in zip(target_columns, [0] * len(target_columns)):
            if col != target:
                row_copy[col] = val

        model = models[target]

        reshaped_row = row_copy.values.reshape(1, -1)
        y_pred = model.predict(reshaped_row)

        print(f"{target} Prediction: {y_pred[0]}")
        
    print("\n")


Teams: Brentford vs Arsenal
Home_Goals Prediction: 1.2188491478941184
Away_Goals Prediction: 0.6789056305652612
Home_total_shots Prediction: 10.811638086347935
Home_corner_kicks Prediction: 3.751338435019736
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7966724414453589
Away_total_shots Prediction: 8.281259931176553
Away_corner_kicks Prediction: 3.385638697509937
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.0580109957098798
Away_red_cards Prediction: 0.0031479782828875585
Home_red_cards Prediction: -0.03478153008317643
Home_fouls Prediction: 3.64466578350096
Away_fouls Prediction: 4.772529871824898



Teams: Burnley vs West Ham
Home_Goals Prediction: 1.2365862675721788
Away_Goals Prediction: 0.6617110739217592
Home_total_shots Prediction: 10.61308218029976
Home_corner_kicks Prediction: 3.734489894243611
Home_ball_possession Prediction: 1.0000000000000022
Home_yellow_cards Prediction: 0.7828831098316831
Away_t

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Home_fouls Prediction: 3.2038874623784537
Away_fouls Prediction: 4.7810912606576235



Teams: Everton vs Manchester United
Home_Goals Prediction: 1.228804405570428
Away_Goals Prediction: 0.6653884290190016
Home_total_shots Prediction: 10.682548742835783
Home_corner_kicks Prediction: 3.708654745434891
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7829963836911411
Away_total_shots Prediction: 8.341774517400784
Away_corner_kicks Prediction: 3.3640886112523742
Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.084461905463387
Away_red_cards Prediction: 0.0011318502571871646
Home_red_cards Prediction: -0.035693773451820954
Home_fouls Prediction: 3.3884686402800908
Away_fouls Prediction: 4.57803763257713



Teams: Fulham vs Wolves
Home_Goals Prediction: 1.2344491333920666
Away_Goals Prediction: 0.6595538872656096
Home_total_shots Prediction: 10.617705204337472
Home_corner_kicks Prediction: 3.7001777558926534
Home_ball_pos

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.0915924542917999
Away_red_cards Prediction: 0.00016127700078440036
Home_red_cards Prediction: -0.03664421925656171
Home_fouls Prediction: 3.247368416554968
Away_fouls Prediction: 4.562078104056345



Teams: Luton vs Crystal Palace
Home_Goals Prediction: 1.219947932896213
Away_Goals Prediction: 0.669986696305697
Home_total_shots Prediction: 10.763493443547066
Home_corner_kicks Prediction: 3.682802023574779
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7837595722780983
Away_total_shots Prediction: 8.328438118450732
Away_corner_kicks Prediction: 3.3228285296913174
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.092862726929487
Away_red_cards Prediction: 0.0022249374380684164
Home_red_cards Prediction: -0.033123954647136064
Home_fouls Prediction: 3.599479463793771
Away_fouls Prediction: 4.368237263517588



Teams: Manchester City vs L

...

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Away_fouls Prediction: 4.505638129661263



Teams: Luton vs Fulham
Home_Goals Prediction: 1.2220971542411407
Away_Goals Prediction: 0.6681448719267911
Home_total_shots Prediction: 10.740537167196548
Home_corner_kicks Prediction: 3.6828371696775624
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7824597428231
Away_total_shots Prediction: 8.335633067825311
Away_corner_kicks Prediction: 3.326934331112552
Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.093945295681082
Away_red_cards Prediction: 0.0018912035255609545
Home_red_cards Prediction: -0.03357571766537833
Home_fouls Prediction: 3.5466201725696838
Away_fouls Prediction: 4.381730745475687



Teams: Manchester City vs West Ham
Home_Goals Prediction: 1.2309657140801709
Away_Goals Prediction: 0.65954759360504
Home_total_shots Prediction: 10.641259214172461
Home_corner_kicks Prediction: 3.6744128992895
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_c

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Away_corner_kicks Prediction: 3.3357628784089903
Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.102699697636887
Away_red_cards Prediction: 0.00042002940039699696
Home_red_cards Prediction: -0.035203807997482475
Home_fouls Prediction: 3.3262310120084306
Away_fouls Prediction: 4.38601143989205



Teams: Sheffield Utd vs Tottenham
Home_Goals Prediction: 1.2266793585551312
Away_Goals Prediction: 0.6592322313277964
Home_total_shots Prediction: 10.668838514560692
Home_corner_kicks Prediction: 3.640065614835759
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.772033258706378
Away_total_shots Prediction: 8.378160280613097
Away_corner_kicks Prediction: 3.2951197413019027
Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.1176897835556021
Away_red_cards Prediction: 0.0007094102811292156
Home_red_cards Prediction: -0.0333585534884172
Home_fouls Prediction: 3.4225712574090323
Away_fouls Predict

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali